# Short Interest Rate Model Calibration with QuantLib-Python

## Hull-White 1 Factor Model

The Hull-White model is a practical exogenous model for fitting market interest rate term structures, described by:

$$ dr_t = (\theta_t - a r_t) \, dt + \sigma \, dW_t $$

Where:
- $a$ \: is the mean reversion constant ;
- $\sigma$ \: is the volatility parameter ;
- $\theta_t$ \: is chosen to fit the input term structure of interest rates.

### Calibration in QuantLib-Python

To calibrate the Hull-White model in QuantLib-Python, use the `JamshidianSwaptionEngine`. This requires setting up the model with appropriate market data and then solving for the best-fit parameters $a$ and $ \sigma $ that minimize the error in pricing known swaptions.

## Black Karasinski Model

The Black Karasinski model is an interest rate model characterized by:

$$ d \ln(r_t) = (\theta_t - a \ln(r_t)) \, dt + \sigma \, dW_t $$

### Calibration in QuantLib-Python

As this model is non-affine, it necessitates the use of the `TreeSwaptionEngie` for calibration, which is versatile enough to handle various non-affine short rate models. The process involves fitting the model to market swaption volatilities by iteratively adjusting $a$ and $\sigma$ .

## G2 ++ Model : A Two-Factor Calibration Example

The G2++ model involves two factors, $x_t$ and $y_t$, which add complexity and accuaracy to the fitting process :

$$ dr_t = \phi(t) + x_t + y_t $$
$$ dx_t = -ax_t \, dt + \sigma \, dW_{t}^{1} $$
$$ dy_t = -by_t \, dt + \eta \, dW_{t}^{2} $$
$$ \langle dW_{t}^{1}, dW_{t}^{2} \rangle = \rho \, dt $$

### Calibration in QuantLib-Python

For calibrating the G2++ model, QuantLib-Python offers several engines including `TreeSwaptionEngine`, `G2SwaptionEngine` and `FdG2SwaptionEngine`. The choice of engine affects both the calibration time and the accuracy of the model fitted. Calibration typically involves using historical data to estimate the parameters $a, b, \sigma, \eta$ and $\rho$, ensuring the model's effectiveness in simulating and predictinf financial analysis and decision-making.


In [14]:
from QuantLib import *
from collections import namedtuple
import math
from pandas import DataFrame

In [15]:
today = Date(19, February, 2002)
settlement = Date(19, February, 2002)
Settings.instance().evaluationDate = today
term_structure = YieldTermStructureHandle(FlatForward(settlement, 0.04875825, Actual365Fixed()))
index = Euribor1Y(term_structure)

In [16]:
CalibrationData = namedtuple("CalibrationData", "start, length, volatility")
data = [CalibrationData(1, 5, 0.1148),
CalibrationData(2, 4, 0.1108),
CalibrationData(3, 3, 0.1070),
CalibrationData(4, 2, 0.1021),
CalibrationData(5, 1, 0.1000 )]

In [17]:
def create_swaption_helpers(data, index, term_structure, engine) :
    swaptions = []
    fixed_leg_tenor = Period(1, Years)
    fixed_leg_daycounter = Actual360()
    floating_leg_daycounter = Actual360()

    for d in data :
        vol_handle = QuoteHandle(SimpleQuote(d.volatility))
        helper = SwaptionHelper(
            Period(d.start, Years),
            Period(d.length, Years),
            vol_handle,
            index,
            fixed_leg_tenor,
            fixed_leg_daycounter,
            floating_leg_daycounter,
            term_structure
        )
        helper.setPricingEngine(engine)
        swaptions.append(helper)

    return swaptions

In [18]:
def calibration_report(swaptions, data):
    columns = [
        "Model Price", "Market Price", "Implied Vol", "Market Vol",
        "Rel Error Price", "Rel Error Vols"
    ]
    report_data = []
    cum_err = 0.0
    cum_err2 = 0.0

    for i, s in enumerate(swaptions):
        model_price = s.modelValue()
        market_vol = data[i].volatility
        black_price = s.blackPrice(market_vol)
        rel_error = model_price / black_price - 1.0
        implied_vol = s.impliedVolatility(model_price, 1e-5, 50, 0.0, 0.50)
        rel_error2 = implied_vol / market_vol - 1.0
        cum_err += rel_error ** 2
        cum_err2 += rel_error2 ** 2
        report_data.append(
            (model_price, black_price, implied_vol, market_vol, rel_error, rel_error2)
        )
    
    print("Cumulative Error Price : %7.5f" % math.sqrt(cum_err))
    print("Cumulative Error Vols : %7.5f" % math.sqrt(cum_err2))

    return DataFrame(report_data, columns = columns, index = ['']*len(report_data))
        

### Calibration of the Hull-White Model

In [19]:
model = HullWhite(term_structure)
engine = JamshidianSwaptionEngine(model)
swaptions = create_swaption_helpers(data, index, term_structure, engine)
optimization_method = LevenbergMarquardt(1.0e-8,1.0e-8,1.0e-8)
end_criteria = EndCriteria(10000, 100, 1e-6, 1e-8, 1e-8)
model.calibrate(swaptions, optimization_method, end_criteria)
a, sigma = model.params()
print("a = %6.5f, sigma = %6.5f" % (a, sigma))

a = 0.04604, sigma = 0.00578


In [20]:
calibration_report(swaptions, data)

Cumulative Error Price : 0.11275
Cumulative Error Vols : 0.11305


,Model Price,Market Price,Implied Vol,Market Vol,Rel Error Price,Rel Error Vols
,0.008755,0.009434,0.106534,0.1148,-0.071929,-0.072000
,0.009647,0.010051,0.106341,0.1108,-0.040168,-0.040246
,0.008643,0.008700,0.106296,0.1070,-0.006584,-0.006584
,0.006476,0.006218,0.106345,0.1021,0.041422,0.041576
,0.003534,0.003319,0.106484,0.1000,0.064543,0.064841


### Calibration of the Black Karasinski

In [21]:
model = BlackKarasinski(term_structure)
engine = TreeSwaptionEngine(model, 100)
swaptions = create_swaption_helpers(data, index, term_structure, engine)
optimization_method = LevenbergMarquardt(1.0e-8,1.0e-8,1.0e-8)
end_criteria = EndCriteria(10000, 100, 1e-6, 1e-8, 1e-8)
model.calibrate(swaptions, optimization_method, end_criteria)
a, sigma = model.params()
print("a = %6.5f, sigma = %6.5f" % (a, sigma))

a = 0.03921, sigma = 0.11642


In [22]:
calibration_report(swaptions, data)

Cumulative Error Price : 0.12073
Cumulative Error Vols : 0.12104


,Model Price,Market Price,Implied Vol,Market Vol,Rel Error Price,Rel Error Vols
,0.008682,0.009434,0.105640,0.1148,-0.079716,-0.079794
,0.009647,0.010051,0.106341,0.1108,-0.040170,-0.040248
,0.008650,0.008700,0.106383,0.1070,-0.005770,-0.005770
,0.006487,0.006218,0.106540,0.1021,0.043327,0.043489
,0.003547,0.003319,0.106884,0.1000,0.068526,0.068845


### Calibration of G2++ Model

In [23]:
model = G2(term_structure);
engine = TreeSwaptionEngine(model, 25)
# engine = G2SwaptionEngine(model, 10, 400)
# engine = FdG2SwaptionEngine(model)
swaptions = create_swaption_helpers(data, index, term_structure, engine)
optimization_method = LevenbergMarquardt(1.0e-8,1.0e-8,1.0e-8)
end_criteria = EndCriteria(1000, 100, 1e-6, 1e-8, 1e-8)
model.calibrate(swaptions, optimization_method, end_criteria)
a, sigma, b, eta, rho = model.params()
print ("a = %6.5f, sigma = %6.5f, b = %6.5f, eta = %6.5f, rho = %6.5f " % (a, sigma, b, eta, rho))

a = 0.04435, sigma = 0.00300, b = 0.03998, eta = 0.00472, rho = 0.03022 


In [24]:
calibration_report(swaptions, data)

Cumulative Error Price : 0.12160
Cumulative Error Vols : 0.12192


,Model Price,Market Price,Implied Vol,Market Vol,Rel Error Price,Rel Error Vols
,0.008670,0.009434,0.105500,0.1148,-0.080936,-0.081014
,0.009649,0.010051,0.106364,0.1108,-0.039962,-0.040039
,0.008648,0.008700,0.106350,0.1070,-0.006074,-0.006074
,0.006487,0.006218,0.106526,0.1021,0.043188,0.043349
,0.003548,0.003319,0.106915,0.1000,0.068834,0.069154


### Plotting Calibrated IR using HW, BK, G2++

In [12]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Model parameters

# Hull-White parameters
a_hw = 0.04604
sigma_hw = 0.00578

# Black-Karasinski parameters
a_bk = 0.03921
sigma_bk = 0.11642

# G2++ parameters
a_g2 = 0.04435 
sigma_g2 = 0.00300
b_g2 = 0.03998
eta_g2 = 0.00472
rho_g2 = 0.0  # Using exact zero for simplicity

# Simulation settings
r0 = 0.01 # Initial short rate
T = 5 # Total time in years
dt = 1/252 # Daily time steps
n_steps = int(T/dt)
n_paths = 100

# Time array for plotting
time = np.linspace(0, T, n_steps)

# Preallocate matrices for rate simulations
rates_hw = np.zeros((n_steps, n_paths))
rates_bk = np.zeros((n_steps, n_paths))
rates_g2 = np.zeros((n_steps, n_paths))
rates_hw[0,:] = r0
rates_bk[0,:] = np.log(r0) # For BK, work with log-rates
rates_g2[0,:] = r0

# Simulation
sqrt_dt = np.sqrt(dt)

for t in range(1, n_steps) :
    dw = np.random.normal(0, sqrt_dt, (n_paths, 2))
    # Hull-hite simulation
    rates_hw[t, :] = rates_hw[t-1, :] + a_hw * (r0 - rates_hw[t-1, :]) * dt + sigma_hw * dw.T[0]

    # Black-Karasinski simulation
    rates_bk[t, :] = rates_bk[t-1, :] + a_bk * (np.log(r0) - rates_bk[t-1, :]) * dt + sigma_bk * dw.T[0]

    # G2++ simulation (simplified as a single factor for illustration)
    rates_g2[t, :] = rates_g2[t-1, :] + a_g2 * (r0 - rates_g2[t-1, :]) * dt + sigma_g2 * dw.T[0] 

# Convert rates for BK from log back to linear scale for plotting
rates_bk = np.exp(rates_bk)

# Create  Plotly subplots
fig = make_subplots(rows=1, cols=3, subplot_titles=("Black-Karasinski", "Hull-White", "G2++"))

# Plotting each path for each model
for i in range(n_paths) :
    fig.add_trace(go.Scatter(x=time, y=rates_bk[:, i], mode='lines', line=dict(width = 1, color = 'red'), showlegend=False), row=1, col=1)
    fig.add_trace(go.Scatter(x=time, y=rates_hw[:, i], mode='lines', line=dict(width = 1, color = 'blue'), showlegend=False), row=1, col=2)
    fig.add_trace(go.Scatter(x=time, y=rates_g2[:, i], mode='lines', line=dict(width = 1, color = 'green'), showlegend=False), row=1, col=3)


# Update layout to arrange subplot
fig.update_layout(title = 'Interest Rate Model Simulations',
                  xaxis_title='Years', yaxis_title='BK Rates',
                  xaxis2_title='Years', yaxis2_title='HW Rates',
                  xaxis3_title='Years', yaxis3_title='G2++ Rates',
                  template='plotly_dark', height=400, width=1250)

fig.show()